# Assessing Model Performance

#### Ben Smith
#### 10/01/2023

This is chiefly aimed at the historical models (initially uncalibrated) and will create nice plots for publishing.

This script will run through the historical simulations and calculate performance stats for the simulations based on their outlet discharge and daily flow records.

Be aware that shifting the dates by 1 day will significantly affect the NSEs. This is an issue if you index the modelled discharges wrongly. I thought that the first value of the discharge timeseries is the value at the start of the simulation, but this does not seem to be the case. When this value is skipped, the NSEs etc. drop.

## Premable

First create a function (using hydroeval) that will calculate the performance stats that you need.

In [ ]:
# --- PACKAGES ---
import hydroeval as he  # pip install hydroeval from inside a conda prompt
import pandas as pd
import numpy as np
import os

# --- PATHS ---
exe_list = pd.read_csv("list_of_catchment_names.csv", index_col=0)


# --- FUNCTIONS ---
# Some of these are taken from SHETRAN Post Simulation Functions on GitHUB.

# --- Calculate Objective Functions for Flows -----------------
def shetran_obj_functions(
    regular_simulation_discharge_path: str,
    recorded_discharge_path: str,
    start_date: str,
    period: list = None,
    recorded_date_discharge_columns: list = None,
    return_flows=False,
    return_period=False):

    """
    Notes:
    - Assumes daily flow data, can be altered within function.
    - Assumes that recorded flows have dates and are regularly spaced, with no gaps.
    - NAs will be skipped from the analysis. NA count will be returned.

    TODO - consider whether you can add code that allows you to take other columns
            from the record so that they can be visualised at the end.

    regular_simulation_discharge_path:  Path to the txt file
    recorded_discharge_path:            Path to the csv file
    start_date:                         The start date of the simulated flows: "DD-MM-YYYY"
    period:                             The period to use (i.e. calibration/validation) as a list of dates:
                                        ["YYY-MM-DD", "YYY-MM-DD"].
                                        Leave blank if you want to use the whole thing.
                                        Leave as single item in list if you want to use until the end of the data.
    recorded_date_discharge_columns:    The columns (as a list) that contain the date and then flow data.
    RETURNS:                            The NSE value as an array.
    """

    # --- Read in the flows for Sim and Rec:
    if recorded_date_discharge_columns is None:
        recorded_date_discharge_columns = ["date", "discharge_vol"]

    flow_rec = pd.read_csv(recorded_discharge_path,
                           usecols=recorded_date_discharge_columns,
                           parse_dates=[recorded_date_discharge_columns[0]])

    # Set the columns to the following so that they are always correctly referenced:
    # (Do not use recorded_date_discharge_columns!)
    flow_rec.columns = ["date", "discharge_vol"]
    flow_rec = flow_rec.set_index('date')

    # Read in the simulated flows:
    flow_sim = pd.read_csv(regular_simulation_discharge_path)
    flow_sim.columns = ["flow"]

    # --- Give the simulation dates:
    flow_sim['date'] = pd.date_range(start=start_date, periods=len(flow_sim), freq='D')
    flow_sim = flow_sim.set_index('date')  # .shift(-1)
    # ^^ The -1 removes the 1st flow, which is the flow before the simulation.
    # ^^ However, doing this seems to offset the flows and also means that the simulation ends a day sooner than expected... I have therefore removed, which significantly boosts the NSE values!

    # --- Resize them to match
    flows = flow_sim.merge(flow_rec, on="date")
    # ^^ Merge removes the dates that don't coincide. Beware missing record data!

    # Select the period for analysis (if given):
    if period is not None:
        if len(period) == 1:
            flows = flows[flows.index >= period[0]]
        if len(period) == 2:
            flows = flows[(flows.index >= period[0]) & (flows.index <= period[1])]

    # --- Do the comparison
    flow_NAs = np.isnan(flows["discharge_vol"])  # The NAs are actually automatically removed

    # Calculate the objective function:
    obj_funs = {"NSE": np.round(he.evaluator(he.nse, flows["flow"], flows["discharge_vol"]), 2),
                "KGE": np.round(he.evaluator(he.kge, flows["flow"], flows["discharge_vol"]), 2),
                "RMSE": np.round(he.evaluator(he.rmse, flows["flow"], flows["discharge_vol"]), 2),
                "PBias": np.round(he.evaluator(he.pbias, flows["flow"], flows["discharge_vol"]), 2)}

    # Print out the % of data that are NA:
    na_number = len(np.arange(len(flow_NAs))[flow_NAs]) / len(flows) * 100
    if na_number>20:
        print(str(round(na_number, 2)) + "% of comparison data are NA")

    if (period is not None) & (return_period):
        obj_funs["period"] = period

    if return_flows:
        obj_funs["flows"] = flows

    return obj_funs


## APM Scenarios

Import the simulated flows that we want to assess from the APM scenarios:

In [ ]:
root_path = "./"

simulation_periods = {"calibration": ['1990-01-01', "1999-12-31"],
                      "validation": ['2000-01-01', "2009-12-31"]}

# List the catchments that didn't complete, based on the Run Completion.xlsx:
incomplete_simulations = ["76011","39061", "80005"]

# Get a list of the catchments to test:
simulations = os.listdir(root_path)
simulations = [s for s in simulations if "." not in s]
simulations = [s for s in simulations if s not in incomplete_simulations]
simulations = sorted(simulations)

In [ ]:
# Create database to hold calibration outputs:
performance_calibration = pd.DataFrame({"simulation":simulations,
                                        "NSE":np.nan, "KGE":np.nan, "KGE_r":np.nan, "KGE_a":np.nan, "KGE_B":np.nan,
                                        "RMSE":np.nan, "PBias":np.nan})
performance_calibration.set_index('simulation', inplace=True)

# Create database to hold validation outputs:
performance_validation = pd.DataFrame({"simulation":simulations,
                                       "NSE":np.nan, "KGE":np.nan, "KGE_r":np.nan, "KGE_a":np.nan, "KGE_B":np.nan,
                                       "RMSE":np.nan, "PBias":np.nan})
performance_validation.set_index('simulation', inplace=True)

performance_calibration

In [ ]:
for s in performance_calibration.index:

    print(s)

    # Change flow record path if NI catchment:
    if int(s) >= 200000:
        flow_record_path =  "I:/SHETRAN_GB_2021/02_Input_Data/nrfa_daily_flows/NI restructured to match CAMELS/GDF_" + s + "_19701001-20150930.csv"
    else:
        flow_record_path = "I:/CAMELS-GB/data/timeseries/CAMELS_GB_hydromet_timeseries_" + s + "_19701001-20150930.csv"

    # --- CALIBRATION:

    try:
        p = shetran_obj_functions(
        regular_simulation_discharge_path=f"{root_path}{s}/output_{s}_discharge_sim_regulartimestep.txt",
        recorded_discharge_path=flow_record_path,
        recorded_date_discharge_columns=["date", "discharge_vol"],
        start_date='01-01-1980', period=simulation_periods["calibration"],
        return_flows=False, return_period=False)

        performance_calibration.loc[s, "NSE"] = p["NSE"][0]
        performance_calibration.loc[s, "KGE"] = p["KGE"][0,0]
        performance_calibration.loc[s, "KGE_r"] = p["KGE"][1,0]
        performance_calibration.loc[s, "KGE_a"] = p["KGE"][2,0]
        performance_calibration.loc[s, "KGE_B"] = p["KGE"][3,0]
        performance_calibration.loc[s, "RMSE"] = p["RMSE"][0]
        performance_calibration.loc[s, "PBias"] = p["PBias"][0]
    except:
        print("Error with catchment ", s)

    # --- VALIDATION:

    try:
        p = shetran_obj_functions(
            regular_simulation_discharge_path=f"{root_path}{s}/output_{s}_discharge_sim_regulartimestep.txt",
            recorded_discharge_path=flow_record_path,
            recorded_date_discharge_columns=["date", "discharge_vol"],
            start_date='01-01-1980', period=simulation_periods["validation"],
            return_flows=False, return_period=False)

        performance_validation.loc[s, "NSE"] = p["NSE"][0]
        performance_validation.loc[s, "KGE"] = p["KGE"][0,0]
        performance_validation.loc[s, "KGE_r"] = p["KGE"][1,0]
        performance_validation.loc[s, "KGE_a"] = p["KGE"][2,0]
        performance_validation.loc[s, "KGE_B"] = p["KGE"][3,0]
        performance_validation.loc[s, "RMSE"] = p["RMSE"][0]
        performance_validation.loc[s, "PBias"] = p["PBias"][0]
    except:
        print("Error with catchment ", s)

In [ ]:
# --- Write the stats to csvs:
performance_calibration.to_csv("./Simulation performance - Historical - Calibration.csv")
performance_validation.to_csv("./Simulation performance - Historical - Validation.csv")

## Autocalibration Scenarios

In [ ]:
simulations = exe_list.index.astype(str)

# Create database to hold calibration outputs:
performance_calibration = pd.DataFrame({"simulation":simulations,
                                        "NSE":np.nan, "KGE":np.nan, "KGE_r":np.nan, "KGE_a":np.nan, "KGE_B":np.nan,
                                        "RMSE":np.nan, "PBias":np.nan})
performance_calibration.set_index('simulation', inplace=True)

# Create database to hold validation outputs:
performance_validation = pd.DataFrame({"simulation":simulations,
                                       "NSE":np.nan, "KGE":np.nan, "KGE_r":np.nan, "KGE_a":np.nan, "KGE_B":np.nan,
                                       "RMSE":np.nan, "PBias":np.nan})
performance_validation.set_index('simulation', inplace=True)

In [ ]:
# Analyse the autocalibration results:

autocal_flowpath = "./Autocal_Historical_Outlet_Discharge/"

# CALIBRATION:
for s in performance_calibration.index:

    print(s)

    sim_path = f"{autocal_flowpath}output_{s}_discharge_sim_regulartimestep.txt"

    # Change flow record path if NI catchment:
    if int(s) >= 200000:
        flow_record_path = "./GaugedDailyFlow_" + s + "_19701001-20150930.csv"
    else:
        flow_record_path = "./CAMELS_GB_hydromet_timeseries_" + s + "_19701001-20150930.csv"

    # --- CALIBRATION ---
    p = shetran_obj_functions(
        regular_simulation_discharge_path=sim_path,
        recorded_discharge_path=flow_record_path,
        recorded_date_discharge_columns=["date", "discharge_vol"],
        start_date='01-01-1980', period=simulation_periods["calibration"],
        return_flows=False, return_period=False)

    performance_calibration.loc[s, "NSE"] = p["NSE"][0]
    performance_calibration.loc[s, "KGE"] = p["KGE"][0,0]
    performance_calibration.loc[s, "KGE_r"] = p["KGE"][1,0]
    performance_calibration.loc[s, "KGE_a"] = p["KGE"][2,0]
    performance_calibration.loc[s, "KGE_B"] = p["KGE"][3,0]
    performance_calibration.loc[s, "RMSE"] = p["RMSE"][0]
    performance_calibration.loc[s, "PBIas"] = p["PBias"][0]

    # --- VALIDATION ---
    p = shetran_obj_functions(
        regular_simulation_discharge_path=sim_path,
        recorded_discharge_path=flow_record_path,
        recorded_date_discharge_columns=["date", "discharge_vol"],
        start_date='01-01-1980', period=simulation_periods["validation"],
        return_flows=False, return_period=False)

    performance_validation.loc[s, "NSE"] = p["NSE"][0]
    performance_validation.loc[s, "KGE"] = p["KGE"][0,0]
    performance_validation.loc[s, "KGE_r"] = p["KGE"][1,0]
    performance_validation.loc[s, "KGE_a"] = p["KGE"][2,0]
    performance_validation.loc[s, "KGE_B"] = p["KGE"][3,0]
    performance_validation.loc[s, "RMSE"] = p["RMSE"][0]
    performance_validation.loc[s, "PBIas"] = p["PBias"][0]

# Write the stats to csvs:
performance_calibration.to_csv("./Outputs/Simulation performance - Autocal_Historical - Calibration.csv")
performance_validation.to_csv("./Outputs/Simulation performance - Autocal_Historical - Validation.csv")